In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_churn = pd.read_csv('Datasets/churn.csv')
df_customer = pd.read_csv('Datasets/customer.csv')
df_orders = pd.read_csv('Datasets/orders.csv')
df_products = pd.read_csv('Datasets/products.csv')
df_promotions = pd.read_csv('Datasets/promotions.csv')
my_df_list = [df_churn, df_customer, df_orders, df_products, df_promotions] 
my_dfs_name = ['df_churn', 'df_customer', 'df_orders', 'df_products', 'df_promotions']

In [3]:
# converting date columns to datetime format
df_churn.FirstPurchaseDate = pd.to_datetime(df_churn.FirstPurchaseDate)
df_churn.LastPurchaseDate = pd.to_datetime(df_churn.LastPurchaseDate)
df_orders.Purchase_Date = pd.to_datetime((df_orders.Purchase_Date))
df_promotions.StartDate = pd.to_datetime(df_promotions.StartDate)
df_promotions.EndDate = pd.to_datetime(df_promotions.EndDate)

In [4]:
df_promotions.head()

,PromotionID,Brand,Category,DiscountPercentage,StartDate,EndDate
0,1,Nike,Accessories,55,2017-12-08,2017-12-15
1,2,Nike,Accessories,30,2017-12-08,2017-12-15
2,3,Nike,Accessories,20,2017-12-01,2017-12-08
3,4,Nike,Accessories,25,2017-12-01,2017-12-08
4,5,Nike,Accessories,20,2017-12-08,2017-12-15


In [10]:
((df_promotions.EndDate-df_promotions.StartDate).dt.days).unique()

array([7, 5], dtype=int64)

In [13]:
df_orders.groupby('CustomerID')['Brand'].count().max()

56

In [17]:
df_orders.head()

,OrderID,CustomerID,ProductID,Purchase_Date,Quantity,Original_Price,Discount,Discounted_Price,PromotionID,Brand,Category,Total_Amount
0,1,1564,7088,2017-09-02,1,110.0,0,110.0,NaN,Zara,Accessories,110.0
1,1,1564,238,2017-09-02,2,60.0,5,57.0,NaN,Puma,Sportswear,114.0
2,1,1564,8669,2017-09-02,1,210.0,15,178.5,NaN,Ralph Lauren,Sleepwear,178.5
3,2,1471,3570,2017-09-02,3,330.0,0,330.0,NaN,Louis Vuitton,Outerwear,990.0
4,2,1471,396,2017-09-02,1,130.0,10,117.0,NaN,Adidas,Sportswear,117.0


In [14]:
agg_features = df_orders.groupby('CustomerID').agg({
    'Brand': ['nunique', lambda x: x.value_counts().idxmax()],
    'PromotionID': 'nunique',
    'OrderID': 'nunique',
    'Total_Amount': ['sum', 'mean'],
    'Quantity': 'sum'
})

# Rename columns for clarity
agg_features.columns = ['Unique_Brands', 'Most_Frequent_Brand', 'Promotion_Usage_Count', 
                        'Order_Count', 'Total_Spent', 'Average_Order_Value', 'Total_Quantity']

# Calculate derived metrics
agg_features['AOV'] = agg_features['Total_Spent'] / agg_features['Order_Count']
agg_features['UPT'] = agg_features['Total_Quantity'] / agg_features['Order_Count']

# Reset index to make CustomerID a column again
agg_features.reset_index(inplace=True)



In [33]:
df_orders.groupby('CustomerID')[['Brand']].value_counts().idxmax()

(594, 'Zara')

In [15]:
agg_features

,CustomerID,Unique_Brands,Most_Frequent_Brand,Promotion_Usage_Count,Order_Count,Total_Spent,Average_Order_Value,Total_Quantity,AOV,UPT
0,1,14,H&M,3,14,10352.5,383.425926,56,739.464286,4.000000
1,2,14,Tommy Hilfiger,4,15,13143.0,423.967742,78,876.200000,5.200000
2,3,11,Louis Vuitton,5,13,9744.0,348.000000,63,749.538462,4.846154
3,4,12,Ralph Lauren,5,17,11965.0,341.857143,68,703.823529,4.000000
4,5,10,Under Armour,4,9,6499.0,406.187500,38,722.111111,4.222222
...,...,...,...,...,...,...,...,...,...,...
1995,1996,8,Tommy Hilfiger,1,8,6871.5,528.576923,33,858.937500,4.125000
1996,1997,12,Ralph Lauren,9,14,10915.5,363.850000,69,779.678571,4.928571
1997,1998,14,Ralph Lauren,3,13,8243.5,329.740000,50,634.115385,3.846154
1998,1999,10,Nike,4,7,6233.0,479.461538,35,890.428571,5.000000
